In [1]:
# coding: utf-8

from api import spotify_api
import json
import datetime
import re
from time import gmtime, strftime
#import git
import os
from shutil import copyfile
import csv
import requests
import re
import xmltodict

settings_file = '/Users/bednar/Repositories/spotify-new-releases/api/settings.txt'
artists_yt_file = '/Users/bednar/Repositories/spotify-new-releases/artists_yt.txt'

max_album_age = 4 #years
max_video_age = 60 #days

In [2]:
# GETTING ACCESS TOKEN FROM REFRESH TOKEN

access_token = spotify_api.refresh_token(settings_file)


Requesting refresh: 
<Response [200]>
200 OK
BQAZINKhSBJBBy3fN2VOQ98L2wSwizH8wJCg8ufBBrEkebAeZVOi_pBP6i0DqzSeWLWId6DN4rjQZ6UTiYZu9FN1HMAq4f7Fl8rtDpY_XhyJAmnMlbaKOkjtG4w-Dg1I_0TzQ5adR1ZHwqVSm6m04vs_5AXiR2TIDGS5z7CvIK8-sQQ6K4LiDJnnU_XRoDX-LOIL9EOKnZdyyBceeoO6nYgV4qj973O0kzEe7h-_BW7S6xZWfRfxR_QkSlbqdw
BQAZINKhSBJBBy3fN2VOQ98L2wSwizH8wJCg8ufBBrEkebAeZVOi_pBP6i0DqzSeWLWId6DN4rjQZ6UTiYZu9FN1HMAq4f7Fl8rtDpY_XhyJAmnMlbaKOkjtG4w-Dg1I_0TzQ5adR1ZHwqVSm6m04vs_5AXiR2TIDGS5z7CvIK8-sQQ6K4LiDJnnU_XRoDX-LOIL9EOKnZdyyBceeoO6nYgV4qj973O0kzEe7h-_BW7S6xZWfRfxR_QkSlbqdw


In [3]:
# GETTING FOLLOWED ARTISTS

spotify_request_url = 'https://api.spotify.com/v1/'

next_request = spotify_request_url + 'me/following?type=artist&limit=30'
artists_list_spotify = []

while(True):
    next_request = next_request + '&access_token=' + access_token.strip()
    artists_json = spotify_api.spotify_request(next_request)

    print('\n')
    artist_items = artists_json['artists']['items']
    for item in artist_items:
        print(item['name'].encode('utf-8'))
        artists_list_spotify.append(item['name'])

    next_request = artists_json['artists']['next']
    
    print('\n')
    if not next_request:
        print("---END---")
        break


Requesting: https://api.spotify.com/v1/me/following?type=artist&limit=30&access_token=BQAZINKhSBJBBy3fN2VOQ98L2wSwizH8wJCg8ufBBrEkebAeZVOi_pBP6i0DqzSeWLWId6DN4rjQZ6UTiYZu9FN1HMAq4f7Fl8rtDpY_XhyJAmnMlbaKOkjtG4w-Dg1I_0TzQ5adR1ZHwqVSm6m04vs_5AXiR2TIDGS5z7CvIK8-sQQ6K4LiDJnnU_XRoDX-LOIL9EOKnZdyyBceeoO6nYgV4qj973O0kzEe7h-_BW7S6xZWfRfxR_QkSlbqdw
200 OK


b'Filter'
b'Acid Drinkers'
b'Maroon 5'
b'Slipknot'
b'Taylor Swift'
b'God Is An Astronaut'
b'Four Stroke Baron'
b'Russian Circles'
b'Serj Tankian'
b'Hollywood Undead'
b'Agnieszka Chylinska'
b'Royal Republic'
b'Deriglasoff'
b'Shakira'
b'Gojira'
b'Pelican'
b'Porcelain Black'
b'Toska'
b'Volbeat'
b'Red Hot Chili Peppers'
b'The Creatures'
b'M.I.A.'
b'Kvelertak'
b'Mercyful Fate'
b'Ellie Goulding'
b'Dirty Honey'
b'Skyharbor'
b'Type O Negative'
b'M\xc3\xb6tley Cr\xc3\xbce'
b'Bonobo'



Requesting: https://api.spotify.com/v1/me/following?type=artist&after=0cmWgDlu9CwTgxPhf403hb&limit=30&access_token=BQAZINKhSBJBBy3fN2VOQ98L2wSwizH8wJCg8ufBBrEkebAeZVOi

In [4]:
artists_list = []
artists_list_csv = []

with open(artists_yt_file) as csvfile:
    readCSV = csv.reader(csvfile, delimiter='\t')
    
    column_names = []
    
    for row in readCSV:
        column_names = row
        break
    
    for row in readCSV:
        if row[0] == 'Artist':
            continue
        else:
            artists_list_csv.append(row[0])
            artists_list.append(dict(zip(column_names, row)))
            
for artist in artists_list_spotify:
    if artist not in artists_list_csv:
        new_artist = {'Artist': artist, 'YT_channel': '', 'Owner': '', 'Confirmed': '', 'Accept': '', 'Reject': '', 'Comments': ''}
        artists_list.append(new_artist)

# Get the feeds

In [5]:
for item in artists_list:
    if item['YT_channel'] == '':
        continue
    else:
        print(item)

{'Artist': 'Slipknot', 'YT_channel': 'UCOJZ1tna8yj8mAEITPkHNCQ', 'Owner': '', 'Confirmed': 'Yes', 'Accept': '', 'Reject': '', 'Comments': ''}
{'Artist': 'Taylor Swift', 'YT_channel': 'UCqECaJ8Gagnn7YCbPEzWH6g', 'Owner': '', 'Confirmed': 'Yes', 'Accept': '', 'Reject': '', 'Comments': ''}
{'Artist': 'Four Stroke Baron', 'YT_channel': 'UCS0q46Mv9JmKKP4v2FpwMXg', 'Owner': 'No', 'Confirmed': 'Yes', 'Accept': '', 'Reject': '', 'Comments': ''}
{'Artist': 'Royal Republic', 'YT_channel': 'UCi6xHR2FPSGnl0HKMo70onw', 'Owner': '', 'Confirmed': 'Yes', 'Accept': '', 'Reject': '', 'Comments': ''}
{'Artist': 'Gojira', 'YT_channel': 'UC34wpCgr3l9cG0RtFdTTS-Q', 'Owner': '', 'Confirmed': 'Yes', 'Accept': '', 'Reject': '', 'Comments': ''}
{'Artist': 'Red Hot Chili Peppers', 'YT_channel': 'UCEuOwB9vSL1oPKGNdONB4ig', 'Owner': '', 'Confirmed': 'Yes', 'Accept': '', 'Reject': '', 'Comments': ''}
{'Artist': 'Kvelertak', 'YT_channel': 'UCevn3WJiZEdse2quTWPEvvQ', 'Owner': '', 'Confirmed': 'Yes', 'Accept': '', 'Re

In [6]:
def yt_request(channel):
    request = "https://www.youtube.com/feeds/videos.xml?channel_id=" + channel
    print("\nRequesting: {0}".format(request))
    
    url = request
    headers = {
    }
    r = requests.get(url, headers=headers)

    print(r.status_code, r.reason)
    resp = r.text
    if r.status_code == 429 or r.status_code == 54:
        print(resp)
        print('Retrying...')
        
        sleep_time = 2
        if r.status_code == 429:
            sleep_time = 2*int(r.headers['Retry-After'])
            
        time.sleep(sleep_time)
        r = requests.get(url, headers=headers)
        print(r.status_code, r.reason)
        resp = r.text
        
    if r.status_code == 404:
        return ""
    return resp

# Parsing XML

In [7]:
#TODO Create some scoring system

def get_entries(resp, days_old, accept = [], reject = [], accept_remainder = True, channel_owner = True, artist_name = ''):
    if resp == "":
        print("PARSING ERROR: Feed does not exist.")
        return [], []
    if channel_owner == False:
        print("WARNING: Multiple bands on this channel.")
    now = datetime.datetime.now()
    
    accept_generic = ["Official Music Video",
                      "Music Video"]
    
    reject_generic = ["Lyrics Video",
                      "Lyric Video",
                      "(Audio)",
                      "Audio Only",
                      "Live",
                      "Interview",
                      "Making of",
                      "Behind the scenes",
                      "Visualizer",
                      "Tour",
                      "Festival"]
    
    parsed_resp = xmltodict.parse(resp)
    dict_entries = parsed_resp["feed"]["entry"]
    
    pattern = '<entry>.*?</entry>'
    xml_entries = re.findall(pattern, resp, re.DOTALL)
    
    entries_list = []
    garbage_list = []
    
    if len(dict_entries) != len(xml_entries):
        print("PARSING ERROR: Different length of parsed XMLs.")
        return entries_list
    
    for i in range(len(dict_entries)):
        entry_published = dict_entries[i]["published"]
        entry_published_dt = datetime.datetime.strptime(entry_published, '%Y-%m-%dT%H:%M:%S%z').replace(tzinfo=None)
            
        # Reject based on date
        if now-entry_published_dt > datetime.timedelta(days=days_old):
            garbage_list.append({dict_entries[i]["title"] : "Older than "+str(days_old)+" days."})
            continue
            
        # Reject NOT INTERESTING BANDS from Producer Channels
        rejected = 0
        if channel_owner == False:
            if len(re.findall(artist_name.lower(), dict_entries[i]["title"].lower())) == 0:
                garbage_list.append({dict_entries[i]["title"] : "Contained not followed artist."})
                rejected = 1
                break
        if rejected == 1:
            continue
            
        # Reject based on SPECIFIC key-phrases
        rejected = 0
        for reject_item in reject:
            if len(re.findall(reject_item.lower(), dict_entries[i]["title"].lower())) > 0:
                garbage_list.append({dict_entries[i]["title"] : "Contained SPECIFIC phrase: "+ reject_item +"."})
                rejected = 1
                break
        if rejected == 1:
            continue
        
        # Accept based on SPECIFIC key-phrases
        accepted = 0
        for accept_item in accept:
            if len(re.findall(accept_item.lower(), dict_entries[i]["title"].lower())) > 0:
                entries_list.append({dict_entries[i]["title"] : xml_entries[i]})
                accepted = 1
                break
        if accepted == 1:
            continue
            
        # Reject based on GENERIC key-phrases
        rejected = 0
        for reject_item in reject_generic:
            if len(re.findall(reject_item.lower(), dict_entries[i]["title"].lower())) > 0:
                garbage_list.append({dict_entries[i]["title"] : "Contained GENERIC phrase: "+ reject_item +"."})
                rejected = 1
                break
        if rejected == 1:
            continue
        
        # Accept based on GENERIC key-phrases
        accepted = 0
        for accept_item in accept_generic:
            if len(re.findall(accept_item.lower(), dict_entries[i]["title"].lower())) > 0:
                entries_list.append({dict_entries[i]["title"] : xml_entries[i]})
                accepted = 1
                break
        if accepted == 1:
            continue
            
        if accept_remainder:
            entries_list.append({dict_entries[i]["title"] : xml_entries[i]})
        else:
            garbage_list.append({dict_entries[i]["title"] : "Throwing away the remainder."})
    
    return entries_list, garbage_list

In [8]:
resp = yt_request("UCi2KNss4Yx73NG0JARSFe0A")
entries_list, garbage_list = get_entries(resp, max_video_age)

print("ACCEPTED:")
for item in entries_list:
    print(list(item.keys())[0])
    
    
print("\nREJECTED:")
for item in garbage_list:
    print(item)


Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCi2KNss4Yx73NG0JARSFe0A
200 OK
ACCEPTED:

REJECTED:
{'Foo Fighters - Everlong (from Skin And Bones, Live in Hollywood, 2006)': 'Contained GENERIC phrase: Live.'}
{'Foo Fighters - What If I Do? (from Skin And Bones, Live in Hollywood, 2006)': 'Contained GENERIC phrase: Live.'}
{'Foo Fighters - Best Of You (from Skin And Bones, Live in Hollywood, 2006)': 'Contained GENERIC phrase: Live.'}
{'Foo Fighters - Times Like These (from Skin And Bones, Live in Hollywood, 2006)': 'Contained GENERIC phrase: Live.'}
{'Foo Fighters - February Stars (from Skin And Bones, Live in Hollywood, 2006)': 'Contained GENERIC phrase: Live.'}
{'Foo Fighters - Big Me (from Skin And Bones, Live in Hollywood, 2006)': 'Contained GENERIC phrase: Live.'}
{"Foo Fighters - Ain't It The Life (from Skin And Bones, Live in Hollywood, 2006)": 'Contained GENERIC phrase: Live.'}
{'Foo Fighters - See You (from Skin And Bones, Live in Hollywood, 2006)': 'Containe

In [9]:
entries_list_global = []
garbage_list_global = []
channels_no = 0

for item in artists_list:
    if item['YT_channel'] == '':
        continue
    else:
        channels_no += 1
        resp = yt_request(item['YT_channel'])
        if item['Owner'] == 'No':
            channel_owner = False
        else:
            channel_owner = True
        entries_list, garbage_list = get_entries(resp, max_video_age, channel_owner = channel_owner, artist_name = item['Artist'])
        entries_list_global = entries_list_global + entries_list
        garbage_list_global = garbage_list_global + garbage_list
        
print("\nAccepted " + str(len(entries_list_global)) + " videos from " + str(channels_no) + " channels.")
print("\nRejected " + str(len(garbage_list_global)) + " videos.")


Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCOJZ1tna8yj8mAEITPkHNCQ
200 OK

Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCqECaJ8Gagnn7YCbPEzWH6g
200 OK

Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCS0q46Mv9JmKKP4v2FpwMXg
200 OK

Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCi6xHR2FPSGnl0HKMo70onw
200 OK

Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UC34wpCgr3l9cG0RtFdTTS-Q
200 OK

Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCEuOwB9vSL1oPKGNdONB4ig
200 OK

Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCevn3WJiZEdse2quTWPEvvQ
200 OK

Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCt-am4x4ZyeohfgM85qFgdQ
200 OK

Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCCndqRyI-5Zl8Mutlq71ASw
200 OK

Requesting: https://www.youtube.com/feeds/videos.xml?channel_id=UCqR8XPrPbjmT5SQKD2dfJdA
200 OK

Requesting: https://www.youtu

In [10]:
published_str = "\n<published>"+str(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))+"+00:00</published>"

In [11]:
# CREATING FEED WITH ACCEPTED VIDEOS

with open('/Users/bednar/Repositories/spotify-new-releases/yt_stub.xml', 'r') as fs:
    feed_stub=fs.read()
    
feed = feed_stub+published_str

for item in entries_list_global:
    feed = feed + "\n" + list(item.values())[0]

with open('/Users/bednar/Repositories/spotify-new-releases/yt_feed.xml', 'w', encoding='utf-8') as f:
    f.write(feed)

In [12]:
# CREATING FEED WITH LIST OF REJECTED VIDEOS

with open('rejected_stub.xml', 'r') as fs:
    feed_stub=fs.read()
    
title = strftime("%Y-%m-%d %H:%M YouTube newest rejects", gmtime())
feed = re.sub('TITLE', title, feed_stub, flags=(re.MULTILINE | re.DOTALL))

description = 'New releases:&lt;br&gt;'
for item in garbage_list_global:
    description = description + '&lt;br&gt;' + list(item.keys())[0].replace('&', 'AND') + ' - ' + list(item.values())[0].replace('&', 'AND') + '\n'
print(description)
#feed = re.sub('DESCRIPTION', description, feed, flags=(re.MULTILINE | re.DOTALL))
feed = feed.replace('DESCRIPTION', description)

with open('rejected_feed.xml', 'w', encoding='utf-8') as f:
    f.write(feed)

New releases:&lt;br&gt;&lt;br&gt;Slipknot - Europe AND UK 2020 Tour Announce - Older than 60 days.
&lt;br&gt;Slipknot - Knotfest Roadshow (Tour Update) - Older than 60 days.
&lt;br&gt;Alexa: Play 'We Are Not Your Kind' by Slipknot (Amazon Music) - Older than 60 days.
&lt;br&gt;Slipknot - Death Because of Death (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - Insert Coin (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - Red Flag (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - Not Long For This World (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - My Pain (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - A Liar's Funeral (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - Orphan (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - What's Next (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - Critical Darling (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - Spiders (Audio) - Older than 60 days.
&lt;br&gt;Slipknot - Birth Of The Cruel (Vertical Video) - Older than 60 days.
&lt;br

In [13]:
# TODO:
#     
# Move everythin to a script
# Add script to cron
# Create simple feed with rejected videos
# 
# Add column saying what to do with remaining videos
# Add logic to take accept/reject keywords and use them from config file
# Add automated YT channel retrieval?
# Add logic for channels shared with other artists

In [14]:
print('d&d'.replace('&','AN\nD'))

dAN
Dd
